# La fréquence d’apparition des mots dans un texte

Le nombre d’apparitions d’un terme dans un document est souvent la première mesure statistique que l’on calcule. Elle permet de déterminer rapidement la probabilité d’apparition d’un mot, autrement dit sa fréquence relative, et de concevoir une matrice d’occurrences, voire de cooccurrences.

## Difficulté de la tâche

Compter des occurrences revient à subdiviser un objet en éléments et à compter le nombre de fois où chaque élément apparaît. Prenons une liste aléatoire de cent nombres entiers entre 0 et 20 et calculons combien de fois apparaît le nombre 9 :

In [ ]:
from random import randint

l = [ randint(0, 20) for n in range(0, 100) ]

print(f"Le nombre 9 apparaît { l.count(9) } fois.")

On peut réaliser une opération similaire en comptabilisant les apparitions de la lettre *e* dans une phrase :

In [ ]:
sent = "En pratique, un pêcheur pêche avec une canne."

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

Python dissocie par défaut le caractère *e* de ses versions accentuée *ê* et majuscule *E*. Plusieurs stratégies peuvent être mises en place pour y remédier :
- remplacer les caractères ;
- additionner les occurrences de chaque cas ;
- normaliser la phrase.

Prenons la dernière stratégie en faisant appel à la méthode `.normalize()` du module `unicodedata`, qui décompose une lettre en ses différents constituants :

In [ ]:
import unicodedata

def normalize(s):
    """Returns the normalized version of a string.

    s -- string to normalize
    """
    normalized_string = str()
    for c in s:
        components = unicodedata.normalize('NFKD', c)
        base = components[0]
        normalized_string += base.lower()

    return normalized_string

print(normalize(sent))

Grâce à la fonction `map()`, on peut obtenir le même résultat plus rapidement :

In [ ]:
sent = map(lambda x: unicodedata.normalize('NFKD', x)[0], sent)
sent = ''.join(sent)

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

Et pour basculer en bas de casse :

In [ ]:
sent = sent.lower()

print(f"La lettre 'e' apparaît { sent.count('e') } fois.")

## De l’importance de préparer les données

La question de compter des occurrences de mots n’est donc pas si anodine qu’elle peut paraître. La qualité du résultat dépend grandement de la définition de l’objectif que l’on se fixe.

Prenons l’exemple du poème *Le dormeur du val* de Rimbaud, pour lequel on souhaite obtenir une liste de tuples de mots trié par leur fréquence d’apparition, puis par ordre alphabétique :
```python
[('il', 6), ('dans', 5), ('la', 5), ('un', 5)…]
```

Chargeons-le dans une variable `text` :

In [ ]:
with open('./data/dormeur-du-val.txt') as file:
    text = file.read()

**1e étape :** découper en une liste de mots.

In [ ]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer("[\w]+")

words = tokenizer.tokenize(text)

**2e étape :** basculer les mots en bas de casse.

In [ ]:
words = list(
    map(
        lambda w:w.lower(),
        words
    )
)

**3e étape :** regrouper les mots par fréquence d’apparition.

In [ ]:
def get_occurrences(tokens):
    """Builds up a dictionary of words and the count of their
    occurrences.

    tokens -- list of tokens
    """

    occurrences = {}
    for token in tokens:
        occurrences.update({
            token: occurrences.get(token, 0) + 1
        })
    return occurrences

occurrences = get_occurrences(words)

**4e étape :** trier le dictionnaire par ordre alphabétique.

In [ ]:
occurrences = sorted(
    occurrences.items(),
    key=lambda x:x[0]
)

**5e étape :** trier la liste de tuples par ordre décroissant de fréquence d’apparition.

In [ ]:
occurrences = sorted(
    occurrences,
    key=lambda x:x[1],
    reverse=True
)

display(occurrences[:10])

## Améliorer le calcul des occurrences

### `defaultdict`

La structure de données `defaultdict` nous permet d’améliorer la constitution du dictionnaire des occurrences :

In [ ]:
from collections import defaultdict

occurrences = defaultdict(int)

for word in words:
    occurrences[word] = occurrences[word] + 1

# sorting
occurrences = sorted(occurrences.items(), key=lambda x:x[0])
occurrences = sorted(occurrences, key=lambda x:x[1], reverse=True)

display(occurrences[:10])

### `Counter`

Il existe toutefois un autre objet du module `collections` qui est encore plus facilement manipulable pour ce genre d’opérations : `Counter`

In [ ]:
from collections import Counter

occurrences = Counter(words)

Il a l’avantage d’embarquer une méthode pour afficher la liste des items les plus fréquents :

In [ ]:
display(occurrences.most_common(10))

### `FreqDist`

Encore mieux, NLTK inclut un ensemble d’outils pour effectuer des calculs statistiques sur un ensemble de données. L’un d’eux est la classe `FreqDist` du module `nltk.probability`, particulièrement destiné au calcul de la distribution de fréquences.

In [ ]:
import nltk
from nltk.probability import FreqDist

occurrences = nltk.FreqDist(words)

Le résultat est déjà sous la forme d’une liste de tuples triée par ordre de fréquence.

In [ ]:
display(occurrences.most_common(10))

Pour afficher sous forme tabulaire sans recourir à une librairie externe (p.ex : `pandas`), utiliser la méthode `.tabulate()` en sélectionnant les items :

In [ ]:
samples = [ w for w,n in occurrences.most_common(10) ]
tab = occurrences.tabulate(cumulative=False, samples=samples)

En prime, la classe prévoit l’affichage d’un diagramme grâce à une implémentation minimale de `matplotlib` :

In [ ]:
plot = occurrences.plot(25, cumulative=False)

Une dernière astuce pour améliorer la qualité de l’affichage sur les écrans *Retina* :

In [ ]:
%config InlineBackend.figure_format='retina'

plot = occurrences.plot(25, cumulative=False)

### `ConditionalFreqDist`

De manière analogue, la classe `ConditionalFreqDist` permet d’organiser la distribution de fréquences selon une condition. Par exemple, si nous voulions comptabiliser le nombre d’occurences d’un mot en fonction de sa longueur (nombre de ses caractères) :

In [ ]:
from nltk.probability import ConditionalFreqDist

occurrences = nltk.ConditionalFreqDist(
    (len(word), word)
    for word in words
)

La méthode `.conditions()` donne accès aux entrées de la condition exprimée (nombre de caractères d’un mot) :

In [ ]:
display(occurrences.conditions())

Il est tout autant possible d’afficher un graphique, en limitant par exemple le nombre d’entrées :

In [ ]:
%config InlineBackend.figure_format='retina'

plot = occurrences.plot(conditions=[3,8], title="Distribution de fréquences des mots de 3 ou 8 caractères")

## La fréquence relative

Le dénombrement des mots dans un texte ne donne qu’une mesure absolue de la présence de chacun, sans rien dire de leur importance. Qu’un terme apparaisse trois mille fois est en soi beaucoup, mais au milieu d’un corpus de trois milliards de mots, il ne pèse guère. D’où la nécessité de toujours considérer un chiffre parmi son environnement.

Calculer la fréquence relative d’un terme revient à diviser le nombre de fois où il apparaît avec la taille du corpus. En probabilités, la formule revient à :

$$
p(w) = \frac{\text{Card}(w)}{\text{Card}(\Omega)}
$$

Reprenons le dénombrement des mots dans *Le dormeur du val* avec l’aide d’un objet `Counter` :

In [ ]:
occurrences = Counter(words)

**Attention !** Comme il s’agit d’un objet de type `dict`, l'expression `len(occurrences)` ne renverra que le nombre de clés dans le dictionnaire. La taille du corpus s’obtient plutôt en effectuant la somme de ses valeurs :

In [ ]:
card_corpus = sum(occurrences.values())

La probabilité de l’événement *verdure*, soit $p(\text{verdure})$, se calcule ensuite directement :

In [ ]:
display(occurrences['verdure'] / card_corpus)

## Vers la représentation numérique d’un texte

### De la matrice d’occurrences…

Le dénombrement des mots dans un texte nous permet d’introduire le concept de matrices, un objet mathématique qui prend la forme d’un tableaux d’éléments organisés en vecteurs colonnes et en vecteurs lignes.

La bibliothèque logicielle *Numpy* permet de les manipuler :

In [ ]:
# basic libraries
import numpy as np
import pandas as pd

# as matrix
data = np.array([
    list(occurrences.keys()),
    list(occurrences.values())
])

# matrix shape
display(data.shape)

Les matrices étant conçues pour accueillir des objets numériques, manipuler des chaînes de caractères n’est pas conseillé. Pour la représentation, il est préférable de recourir à la bibliothèque *Pandas* :

In [ ]:
# a more suitable way to print a data frame
df = pd.DataFrame(data=data[1].reshape(1, -1), columns=data[0])

display(df)

De là, il est recommandé de séparer les en-têtes et les valeurs sous formes de liste et de matrice :

In [ ]:
words = df.columns
occurrences = df.to_numpy(dtype=int)

Il est toujours possible de récupérer l’information en utilisant l’indice comme moyen de jointure :

In [ ]:
# index of word 'dans' ?
dans = words.get_loc('dans')

# 'dans' frequency
display(occurrences[0][dans])

### … à la matrice de cooccurrences

En gardant trace de l’apparition conjointe de plusieurs événements, une matrice de cooccurrences permet de décrire les relations entre les éléments d’un ensemble.

Considérons un corpus restreint de trois documents :

In [ ]:
corpus = [
    'Le petit chat boit du lait.',
    'Le petit chien boit de l’eau',
    'La vache, comme le chien, boit de l’eau.'
]

Et transformons-le rapidement grâce à une bibliothèque spécialisée en une matrice d’occurrences :

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

display(
    pd.DataFrame(
        data=X.toarray(),
        columns=vectorizer.get_feature_names_out())
)

En algèbre linéaire, la matrice de cooccurrences est simplement le produit de la transposée de la matrice d’occurrences avec elle-même :

In [ ]:
m = X.toarray()
coocc = m.T.dot(m)
np.fill_diagonal(coocc, 0)

display(
    pd.DataFrame(
        data=coocc,
        columns=vectorizer.get_feature_names_out(),
        index=vectorizer.get_feature_names_out())
)